In [482]:
import pandas as pd
import numpy as np
import random

In [483]:
sh = []
with open('sh.txt', 'r', encoding='utf-8') as outfile:
    for i in outfile.readlines():
        if i != '\n':
            i = i.rstrip()
            sh.append(i)
        

In [484]:
data_sh = pd.DataFrame(sh, columns=['text'])

In [485]:
data_sh['class'] = 1

In [486]:
pushkin = []
with open('Pushkin.txt', 'r', encoding='utf-8') as outfile:
    for i in outfile.readlines():
        if i != '\n':
            i = i.strip()
            pushkin.append(i)
        

In [487]:
pushkin = pushkin[:245]

In [488]:
data_pushkin = pd.DataFrame(pushkin, columns=['text'])

In [489]:
data_pushkin['class'] = 0

In [490]:
stih=[]
with open('stih.txt', 'r', encoding='utf-8') as outfile:
    for i in outfile.readlines():
        if i != '\n':
            i = i.strip()
            stih.append(i)

In [491]:
def addtext(text):
    text = f'{random.choice(stih)} {text}'
    return text

In [492]:
data_pushkin['text'] = data_pushkin['text'].apply(addtext)

In [493]:
df= data_pushkin.append(data_sh, ignore_index=True)

C:\Users\user\AppData\Local\Temp\ipykernel_12316\3648041625.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df= data_pushkin.append(data_sh, ignore_index=True)


In [494]:
df = df.sample(frac=1).reset_index(drop=True)

In [495]:
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
import transformers

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert = TFAutoModel.from_pretrained("bert-base-cased")

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [496]:
SEQ_LEN=30

In [497]:
arr = df['class'].values  # take sentiment column in df as array
labels = np.zeros((arr.size, arr.max()+1))  # initialize empty (all zero) label array
labels[np.arange(arr.size), arr] = 1  # add ones in indices where we have a value

# define function to handle tokenization
def tokenize(sentence):
    tokens = tokenizer.encode_plus(sentence, max_length=30,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_attention_mask=True,
                                   return_token_type_ids=False, return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

# initialize two arrays for input tensors
Xids = np.zeros((len(df), SEQ_LEN))
Xmask = np.zeros((len(df), SEQ_LEN))

# loop through data and tokenize everything
for i, sentence in enumerate(df['text']):
    Xids[i, :], Xmask[i, :] = tokenize(sentence)

# create tensorflow dataset object
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

# restructure dataset format for BERT
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels
  
dataset = dataset.map(map_func)  # apply the mapping function

# shuffle and batch the dataset
dataset = dataset.shuffle(10000).batch(32)

DS_LEN = len(list(dataset))  # get dataset length

SPLIT = 0.9  # we will create a 90-10 split

# create training-validation sets
train = dataset.take(round(DS_LEN*SPLIT))
val = dataset.skip(round(DS_LEN*SPLIT))


In [498]:
# build the model
input_ids = tf.keras.layers.Input(shape=(30,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(30,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[0]  # we only keep tensor 0 (last_hidden_state)

X = tf.keras.layers.GRU(128)(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(64, activation='relu')(X)
X = tf.keras.layers.Dropout(0.7)(X)
y = tf.keras.layers.Dense(2, activation='sigmoid',  name='outputs')(X)  # adjust based on number of sentiment classes

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# freeze the BERT layer
model.layers[2].trainable = False

# compile the model
optimizer = tf.keras.optimizers.Adam(0.001)
loss = tf.keras.losses.BinaryCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [499]:
history = model.fit(dataset, epochs=10)

Epoch 1/10
16/16 [==============================] - 13s 66ms/step - loss: 0.7693 - accuracy: 0.6612
Epoch 2/10
16/16 [==============================] - 1s 65ms/step - loss: 0.5011 - accuracy: 0.8000
Epoch 3/10
16/16 [==============================] - 1s 65ms/step - loss: 0.3835 - accuracy: 0.8673
Epoch 4/10
16/16 [==============================] - 1s 65ms/step - loss: 0.3173 - accuracy: 0.8857
Epoch 5/10
16/16 [==============================] - 1s 65ms/step - loss: 0.2590 - accuracy: 0.9184
Epoch 6/10
16/16 [==============================] - 1s 65ms/step - loss: 0.2266 - accuracy: 0.9143
Epoch 7/10
16/16 [==============================] - 1s 64ms/step - loss: 0.1616 - accuracy: 0.9510
Epoch 8/10
16/16 [==============================] - 1s 64ms/step - loss: 0.1457 - accuracy: 0.9429
Epoch 9/10
16/16 [==============================] - 1s 64ms/step - loss: 0.1493 - accuracy: 0.9531
Epoch 10/10
16/16 [==============================] - 1s 64ms/step - loss: 0.1227 - accuracy: 0.9694


In [512]:
Xids, Xmask = tokenize("Привет, твои как дела?")

In [513]:
test = {'input_ids': Xids, 'attention_mask': Xmask}

In [514]:
model(test)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.4714664 , 0.43188226]], dtype=float32)>

In [503]:
np.argmax(model(test)[0])

1

In [511]:
Xids, Xmask = tokenize("Что сказал Путин?")
test = {'input_ids': Xids, 'attention_mask': Xmask}
model(test)


<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.1289255 , 0.81518775]], dtype=float32)>

In [505]:
np.argmax(model(test)[0])

0

In [506]:
np.max(model(test)[0])

0.4004165

In [507]:
Xids, Xmask = tokenize("Помоги продолжить стих: Пойдем покатаемся")
test = {'input_ids': Xids, 'attention_mask': Xmask}
model(test)


<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.9723932 , 0.01688579]], dtype=float32)>

In [508]:
model.save('models/first_classif.h5')

In [509]:
first_classif_model = tf.keras.models.load_model('models/first_classif.h5', custom_objects={"TFBertModel": transformers.TFBertModel})

In [510]:
Xids, Xmask = tokenize("Помоги сочинить стих: Природа любит")
test = {'input_ids': Xids, 'attention_mask': Xmask}
first_classif_model(test)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.9330265, 0.030246 ]], dtype=float32)>